# Random forest with Christman Field Data
[![Latest release](https://badgen.net/github/release/Naereen/Strapdown.js)](https://github.com/eabarnes1010/course_objective_analysis/tree/main/code)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eabarnes1010/course_objective_analysis/blob/main/code/random_forest_christman.ipynb)

* Iris example adapted from: https://www.datacamp.com/community/tutorials/random-forests-classifier-python
* Further modified by: Aaron Hill and Wei-Ting Hsiao (Dept. of Atmospheric Science, Colorado State University), January 2020
* Further adapted by: Prof. Elizabeth Barnes for ATS 655 and ATS 780A7 Spring 2022 at Colorado State University

Lets import some libraries we will need throughout this tutorial:



In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
print('IN_COLAB = ' + str(IN_COLAB))

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import datetime
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.inspection import permutation_importance
import pydot
import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.rcParams["figure.facecolor"] = "white"
mpl.rcParams["figure.dpi"] = 150


In [ ]:
print(f"python version = {sys.version}")
print(f"numpy version = {np.__version__}")
print(f"scikit-learn version = {sklearn.__version__}")  

In [ ]:
if(IN_COLAB==True):
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        local_path = '/content/drive/My Drive/Colab Notebooks/'
    except:
        local_path = './'
else:
    local_path = '../figures/'

In [ ]:
FS = 16
# plt.rc('text',usetex=False)
# plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']}) 
plt.rc('savefig',facecolor='white')
plt.rc('axes',facecolor='white')
plt.rc('axes',labelcolor='dimgrey')
plt.rc('axes',labelcolor='dimgrey')
plt.rc('xtick',color='dimgrey')
plt.rc('ytick',color='dimgrey')


# 1. Data Preparation

### 1.1 Data overview

We have stored a .csv file on a CSU drive, accessible via URL. This will be the basis for our tutorial. This file contains Fort Collins weather data from 2020, and we will use these data to predict the high temperature for a given day with a random forest regression model.  

In [ ]:
# Read in data from url
url = "https://raw.githubusercontent.com/eabarnes1010/course_ml_ats/main/data/fccwx_data_2020.csv"
data = pd.read_csv(url,parse_dates=["Date"],infer_datetime_format=True)
data['dayofyear'] = data['Date'].dt.dayofyear
data.reindex(index=data.index[::-1])

Lets look at our data to see what we are working with

In [ ]:
# Display first 5 rows
print('The shape of our features is:', data.shape)

In [ ]:
# A handy tool in pandas: descriptive statistics for each column
data.describe()

### 1.2 Targets and features

The pandas table is handy for a quick glance, but we need to organize some numpy arrays that separately contain our features and labels.

In [ ]:
THRESHOLD_TEMP = 10   # default = 10
TARGET_VAR = 'Temp [C]'

# Labels are the values we want to predict
labels = (data[TARGET_VAR] > THRESHOLD_TEMP).astype(int) + (data[TARGET_VAR] > -THRESHOLD_TEMP/2).astype(int)
label_names = ('really cold <' + str(THRESHOLD_TEMP) + 'C', 'cold <' + str(THRESHOLD_TEMP) + 'C', 'warm >' + str(THRESHOLD_TEMP) + 'C')

# Remove the labels from the features
# axis 1 refers to the columns
features = data.drop(TARGET_VAR, axis = 1)

# Also remove DewPt and Date
features = features.drop('DewPt [C]', axis = 1)   # MODIFY: comment out if you want the prediction task to be easy
features = features.drop('Date', axis = 1)

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features.describe()

As a best practice for classification tasks, always look at a histograms of your labels prior to moving on.

In [ ]:
# convert predictors to a numpy array
features = np.array(features)

In [ ]:
plt.figure()
plt.hist(labels)
plt.xticks(np.arange(0,len(label_names)),label_names)
plt.xlim(-1,3)
plt.title('Sample counts by label')
plt.ylabel('sample count')
plt.show()

### 1.3 Splitting training and testing datasets

Assuming we have no feature data available from 2019 we could use to test our trained models against, we will want to split up our dataset into training and testing portions. A standard proportion is 3/4 for training, 1/4 for testing, although this is somewhat arbitrary here. 

In [ ]:
# Split the data into training and testing sets
split_size = 0.25  # MODIFY: proportion of the dataset we want to use for testing. 1 - split_size is used for training. 

# PARAMETERS:
#     test_size: fraction of testing/validation datasets
#     random_state: random parameter
train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size = split_size, random_state = 42)

Lets quickly check the size of our training and testing arrays are what we expect (and we didn't do something wrong)

In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Validation Features Shape:', val_features.shape)
print('Validation Labels Shape:', val_labels.shape)

In [ ]:
fig, ax1 = plt.subplots(figsize=(7.5,3))
plt.plot(features[:,-1],np.array(data[TARGET_VAR]),'.')
plt.axhline(y=THRESHOLD_TEMP,linestyle='--',color='r')
plt.axhline(y=-THRESHOLD_TEMP/2,linestyle='--',color='r')
plt.xlabel('day of year')
plt.ylabel(TARGET_VAR)
plt.show()

# 2. Creating a random forest

### Train the model and visualize it

In [ ]:
# RANDOM FOREST
tree_depth = 1       # MODIFY: how deep the tree is allowed to go 
node_split = 2       # MODIFY: minimum number of training samples needed to split a node
leaf_samples = 2     # MODIFY: minimum number of training samples required to make a leaf node
number_of_trees = 1  # MODIFY: number of trees to pool/average together in your random forest
criterion = 'gini'   # MODIFY: method for choosing impurity, can be "gini" or "entropy"
RAND_STATE = 42      # MODIFY: random number to split data up

tree_clf = RandomForestClassifier(n_estimators = number_of_trees, 
                           random_state = RAND_STATE,
                           min_samples_split = node_split,
                           min_samples_leaf = leaf_samples,
                           criterion = criterion,
                           max_depth = tree_depth)
tree_clf.fit(train_features,train_labels)

#----------------------------------------------------------------------------------------------------
from graphviz import Source
fig_savename = 'tree_classifier_christman'

if(len(np.shape(tree_clf))!=0):
    tree_to_viz = tree_clf[0]       # MODIFY: which tree in your random forest do you want to plot, default is the first, i.e. [0]
else:
    tree_to_viz = tree_clf
export_graphviz(tree_to_viz,
                out_file=local_path + fig_savename+'.dot',
                filled=True,
                proportion=False,
                leaves_parallel=False,
                class_names=label_names,
                feature_names=feature_list)
Source.from_file(local_path + fig_savename+'.dot')

### Make predictions

In [ ]:
# predict the class for each sample 
# result will be an integer for each prediction
y_pred_train = tree_clf.predict(train_features)
y_pred_val = tree_clf.predict(val_features)
y_pred_val

In [ ]:
# show the predicted proababilities for each class
# result will be a row for each class, and each row will
# have three columns denoting the Pr() for each type of iris
tree_clf.predict_proba(val_features)[:5]

### Visualization of predictions

In [ ]:
for split_type in ("Training","Validation"):
    
    if split_type=="Training":
        plot_features = train_features
        plot_pred = y_pred_train
    elif split_type=="Validation":
        plot_features = val_features
        plot_pred = y_pred_val        
    else:
        raise NotImplementedError()
    
    fig, ax1 = plt.subplots(figsize=(7.5,3))
    ax2 = ax1.twinx()

    ax1.plot(features[:,-1],np.array(data[TARGET_VAR]),'.')
    ax1.set_xlabel('day of year')
    ax1.set_ylabel(TARGET_VAR)

    ax2.plot(plot_features[:,-1],plot_pred,'s',markersize=3,alpha=.3,linewidth=.1,markeredgecolor='r',markerfacecolor='None',)
    ax2.set_yticks(np.arange(0,len(label_names)))
    ax2.set_yticklabels(label_names, color='r')
    ax2.set_ylabel('Random Forest Predicted Class',color='r')

    plt.title(split_type + ' Predictions')
    plt.show()

### Evaluate the classification predictions

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

for split_type in ("Training","Validation"):
    print(split_type + ' confusion matrix')    
    
    if split_type=="Training":
        plot_pred = tree_clf.predict(train_features)
        plot_labels = train_labels
    elif split_type=="Validation":
        plot_pred = tree_clf.predict(val_features)
        plot_labels = val_labels
    else:
        raise NotImplementedError()

    print(confusion_matrix(plot_labels, plot_pred))
    
    plt.figure()
    ConfusionMatrixDisplay.from_predictions(plot_labels, plot_pred,normalize='true')
    plt.title(split_type + ' Data')
    plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

print('ACCURACY')
print('    Training   = ' + str((accuracy_score(train_labels, y_pred_train)).round(3)))
print('    Validation = ' + str((accuracy_score(val_labels, y_pred_val)).round(3)))

print('')

print('Macro F1-Score')
print('    Training   = ' + str((f1_score(train_labels, y_pred_train, average='macro')).round(3)))
print('    Validation = ' + str((f1_score(val_labels, y_pred_val, average='macro')).round(3)))

print('')

print('Weighted F1-Score')
print('    Training   = ' + str((f1_score(train_labels, y_pred_train, average='weighted')).round(3)))
print('    Validation = ' + str((f1_score(val_labels, y_pred_val, average='weighted')).round(3)))


# EVALUATE YOUR MODEL ON 2021 TESTING DATA

In [ ]:
raise ValueError('do not go below this line!')

In [ ]:
# # Read in data from url
from sklearn.metrics import f1_score

url = "https://raw.githubusercontent.com/eabarnes1010/course_ml_ats/main/data/fccwx_data_2021.csv"
data = pd.read_csv(url,parse_dates=["Date"],infer_datetime_format=True)
data['dayofyear'] = data['Date'].dt.dayofyear
data.reindex(index=data.index[::-1])

# Labels are the values we want to predict
labels = (data[TARGET_VAR] > THRESHOLD_TEMP).astype(int) + (data[TARGET_VAR] > -THRESHOLD_TEMP/2).astype(int)

# Remove the labels from the features
# axis 1 refers to the columns
features = data.drop(TARGET_VAR, axis = 1)

# Also remove DewPt and Date
features = features.drop('DewPt [C]', axis = 1)
features = features.drop('Date', axis = 1)

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

# make the predictions
y_pred_test = tree_clf.predict(features)

# print the metrics report
print(sklearn.metrics.classification_report(labels,y_pred_test))

# print final f1 score
print('---------------------------------------')
print('Testing Macro F1-Score   : ' + str((f1_score(labels, y_pred_test,average='macro')).round(3)))
print('Testing Weighted F1-Score: ' + str((f1_score(labels, y_pred_test,average='weighted')).round(3)))

# print accuracies
print('---------------------------------------')
print('TRAINING ACCURACY  : ' + str((accuracy_score(train_labels, y_pred_train)).round(3)))
print('VALIDATION ACCURACY: ' + str((accuracy_score(val_labels, y_pred_val)).round(3)))
print('TESTING ACCURACY   : ' + str((accuracy_score(labels, y_pred_test)).round(3)))


fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(features[:,-1],np.array(data[TARGET_VAR]),'.')
ax1.set_xlabel('day of year')
ax1.set_ylabel(TARGET_VAR)

ax2.plot(features[:,-1],y_pred_test,'s',markersize=3,alpha=.3,linewidth=.1,markeredgecolor='r',markerfacecolor='None',)
ax2.set_yticks(np.arange(0,len(label_names)))
ax2.set_yticklabels(label_names, color='r')
ax2.set_ylabel('Random Forest Predicted Class',color='r')

plt.title('Testing Predictions - 2021')
plt.show()

plt.figure()
ConfusionMatrixDisplay.from_predictions(labels, y_pred_test, normalize='true')
plt.title('Testing Data')
plt.show()
